In [150]:
import numpy as np
import math as m
import scipy as sp

In [104]:
class NormalRV:
    def __init__(self, mu, Sigma):
        self.mu = mu
        self.Sigma = Sigma
        self.S = np.linalg.inv(Sigma)
        self.m = self.S@self.mu
    
    def pdf(self, x):
        # x a numpy array
        # should work if x is a matrix, applying the pdf to each column
        mu = self.mu
        Sigma = self.Sigma
        d = len(mu)
        x = np.asarray(x)
        if x.ndim == 1:
            x = x[:, np.newaxis]
        n = x.shape[1]
        pdf = np.zeros(n)
        for i in range(n):
            pdf[i] = 1/np.sqrt((2*np.pi)**d * np.linalg.det(Sigma)) * \
                np.exp(-1/2 * (x[:,i] - mu).T @ np.linalg.inv(Sigma) @ (x[:,i] - mu))
        return pdf

    def sample(self, n):
        # n: number of samples
        mu = self.mu
        Sigma = self.Sigma
        return np.random.multivariate_normal(mu, Sigma, n).T
    
    def update_vals(self, m, S):
        self.m = m
        self.S = S
        self.Sigma = np.linalg.inv(S)
        self.mu = self.Sigma@m
        

In [105]:
q = NormalRV(np.array([0,0]), np.array([[1,0],[0,1]]))
pi = NormalRV(np.array([1,1]), np.array([[1,0],[0,1]]))




In [106]:
def partial_lq_m(x, p):
    return x - np.reshape(p.mu, (len(p.mu), 1))

def partial_lq_S(x, p):
    # x can be a matrix, each column is a sample
    # in this case, return a 3D array, each slice is a matrix  
    # (i.e. a 2D array) corresponding to a sample
    x = np.asarray(x)
    if x.ndim == 1:
        x = x[:, np.newaxis]
    n = x.shape[1]
    d = len(p.mu)
    res = np.zeros((d,d,n))
    for i in range(n):
        xi = x[:,i]
        res[:,:,i] = -0.5*(np.outer(xi, xi) - np.outer(p.mu, p.mu)- p.Sigma)
    return res


In [107]:
def is_pd(M):

    # efficient check if a matrix is positive-definite
    # from https://stackoverflow.com/questions/16266720/find-out-if-matrix-is-positive-definite-with-numpy

    try:
        _ = np.linalg.cholesky(M)
        return True
    except np.linalg.LinAlgError:
        return False

In [108]:
def project_pd(M, eps=1e-6):
    # project a matrix to the positive definite cone
    # M: a symmetric matrix
    # return: a symmetric matrix
    if is_pd(M):
        return M

    # eigen decomposition
    eig_vals, eig_vecs = np.linalg.eig(M)
    # set negative eigenvalues to eps
    eig_vals[eig_vals <= 0] = eps
    # reconstruct matrix
    return eig_vecs @ np.diag(eig_vals) @ eig_vecs.T


In [153]:
def OAIS(phi, pi, q0, lr, nsamples, niter):
    results = []
    for i in range(niter):
        # compute inner product
        samples = q0.sample(nsamples)
        w = pi.pdf(samples) / q0.pdf(samples)
        w2 = w**2
        phi_samples = np.apply_along_axis(phi, 0, samples)
        integral = np.mean(w*phi_samples)/np.mean(w)

        # update q0
        results.append(integral)
        partial_m = partial_lq_m(samples, q0)
        update_m = -np.mean(w2 * partial_m, axis=1)
        
        partial_S = partial_lq_S(samples, q0)
        update_S = -np.mean(w2*partial_S, axis=2)
        
        new_S = project_pd(q0.S + lr*update_S)
        new_m = q0.m + lr*update_m
        q0.update_vals(new_m, new_S)

    return results
        
        
        
q = NormalRV(np.array([1,1]), np.array([[1,0],[0,1]]))
pi = NormalRV(np.array([0,0]), np.array([[1,0],[0,1]]))

def phi(x):
    if np.abs(x[0]) < 0.5 and np.abs(x[1]) < 0.5:
        return 1
    else:
        return 0
e = OAIS(phi, pi, q, 0.001, 1000, 100)
e

v = sp.stats.norm.cdf(0.5) - sp.stats.norm.cdf(-0.5)
np.abs(e - v**2)


/var/folders/82/t8qzqs6x2f19402hdk543xd80000gn/T/ipykernel_41368/469297332.py:9: RuntimeWarning: invalid value encountered in double_scalars
  integral = np.mean(w*phi_samples)/np.mean(w)


array([0.05357321, 0.02769911, 0.00856259, 0.01229092, 0.01493629,
       0.01246199, 0.02993752, 0.01543226, 0.00386318, 0.03396258,
       0.02786853, 0.04218538, 0.0268192 , 0.00087089, 0.0533944 ,
       0.03151307, 0.00648567, 0.00138014, 0.01964687, 0.07567072,
       0.10349292,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        

In [146]:
m.erf(-0.5)

-0.5204998778130465